## 1. IMPORTING ALL DEPENDANCIES

In [ ]:
import os

import lightgbm as lgb
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import pandas as pd
import seaborn as sns
from catboost import CatBoostRegressor
from scipy import stats
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.ensemble import (
    GradientBoostingRegressor,
    RandomForestRegressor,
    StackingRegressor,
    VotingRegressor,
)
from sklearn.impute import SimpleImputer
from sklearn.linear_model import ElasticNet, Lasso, LinearRegression, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from xgboost import XGBRegressor

## 2. LOAD AND PREPARE DATA

In [ ]:
housing_df = pd.read_csv(
    "/home/kobey/Documents/DATASCIENCE/PROJECTS/CALIFORNIA HOUSING PRICES/data/01-raw/housing.csv"
)
housing_df.shape

In [ ]:
housing_df.columns

In [ ]:
housing_df.describe()

In [ ]:
msno.matrix(housing_df)

## 3. INITIAL EXPLORATION

In [ ]:
housing_df.dtypes

In [ ]:
housing_df.isnull().sum()

#### Dealing with missing data

In [ ]:
housing_df = housing_df.dropna()

In [ ]:
msno.matrix(housing_df)

## 4. Preprocessing

In [ ]:
housing_df["ocean_proximity"].value_counts()

#### Shuffling the data

In [ ]:
housing_df_shuffled = housing_df.sample(n=len(housing_df), random_state=1)
housing_df_shuffled

#### convert categorical variables into float types

In [ ]:
pd.get_dummies(housing_df_shuffled["ocean_proximity"]).head()

#### drop ocean proximity column

In [ ]:
housing_df_shuffled.drop("ocean_proximity", axis=1).head()

#### Apply the two above to our DataFrame

In [ ]:
final_housing_df = pd.concat(
    [
        housing_df_shuffled.drop("ocean_proximity", axis=1),
        pd.get_dummies(housing_df_shuffled["ocean_proximity"]),
    ],
    axis=1,
)

In [ ]:
final_housing_df.columns

#### Restructure the columns, the dependent variable median_house_value(y) should be the last column

In [ ]:
final_housing_df = final_housing_df[
    [
        "longitude",
        "latitude",
        "housing_median_age",
        "total_rooms",
        "total_bedrooms",
        "population",
        "households",
        "median_income",
        "<1H OCEAN",
        "INLAND",
        "ISLAND",
        "NEAR BAY",
        "NEAR OCEAN",
        "median_house_value",
    ]
]

In [ ]:
final_housing_df

In [ ]:
len(final_housing_df)

#### Saving this current dataframe to preprocessed folder

In [ ]:
# Define your folder and filename
folder_path = (
    "/home/kobey/Documents/DATASCIENCE/PROJECTS/CALIFORNIA HOUSING PRICES/data/02-preprocessed"
)
file_name = "preprocessed.csv"
full_path = os.path.join(folder_path, file_name)

# Create the directory if it doesn't exist (optional but recommended)
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Save the DataFrame to the specified path
final_housing_df.to_csv(full_path, index=False)  # index=False prevents writing row numbers